In [8]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [9]:
fre_mulitplier = 78.5

countries = ['bangladesh', 'brazil', 'cambodia', 'china', 'indonesia', 'mexico','rwanda','kenya']

city_country = ['brazil', 'cambodia','rwanda']

def convert_to_flt(str):
    try:
        return float(str.replace(",", ""))  
    except ValueError:
        return None

In [10]:
def make_df_admin(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    rows = soup.find_all('tr', class_=['rname', 'admin2'])
    # rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')
    # import pdb; pdb.set_trace()
    years = []
    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        # import pdb; pdb.set_trace()
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            years.append(year)


        
    for row in rows:
        try:
            # Get the name, status, and population for each census year
            name = row.find('td', class_='rname').text.strip()
            status = row.find('td', class_='rstatus').text.strip()

            # Extract area and population data for different census years
            # import pdb; pdb.set_trace()
            area = convert_to_flt(row.find('td', class_='rname')['data-area'])  # Get the 'data-area' attribute

            # Extract all population data (there could be multiple columns for different years)
            population_cells = row.find_all('td', class_='rpop')

            # Create a dictionary to store population data for each year
            population_data = {}
            for i, cell in enumerate(population_cells):
                population = cell.text.strip()
                population_data[f'Population {years[i]}'] = convert_to_flt(population)

            # Append the extracted data into the list
            row_data = [name, status, area] + list(population_data.values())
            data.append(row_data)
            
        except:
            continue
    # Create a DataFrame
    columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column

In [3]:
def make_df_city(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    # rows = soup.find_all('tr', class_=['rname', 'admin2'])
    rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')

    years = []

    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            if year not in years:
                years.append(year)

        
    for row in rows:
        try:
            # Get the name, status, and population for each census year
            name = row.find('td', class_='rname').text.strip()
            status = row.find('td', class_='rstatus').text.strip()

            # Extract area and population data for different census years
            # import pdb; pdb.set_trace()
            area = row.find('td', class_='rarea').text.strip()  # Get the 'data-area' attribute

            # Extract all population data (there could be multiple columns for different years)
            population_cells = row.find_all('td', class_='rpop')

            # Create a dictionary to store population data for each year
            population_data = {}
            for i, cell in enumerate(population_cells):
                population = cell.text.strip()
                population_data[f'Population {years[i]}'] = convert_to_flt(population)

            # Append the extracted data into the list
            row_data = [name, status, area] + list(population_data.values())
            data.append(row_data)
            
        except Exception as e:
            import pdb; pdb.set_trace()
            continue
    # Create a DataFrame
    columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column

In [4]:
def make_df_city(page, section_id):
    soup = BeautifulSoup(page.content, 'html.parser')
    soup = soup.find(id=section_id)
    # print(soup.prettify())

    # rows = soup.find_all('tr', class_=['rname', 'admin2'])
    rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')

    years = []

    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            if year not in years:
                years.append(year)

    # import pdb; pdb.set_trace()
        
    for row in rows:
        if section_id == 'adminareas':
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', class_='rname').text.strip()
                status = row.find('td', class_='rstatus').text.strip()

                # Extract area and population data for different census years
                # import pdb; pdb.set_trace()
                area = row.find('td', class_='rarea').text.strip()  # Get the 'data-area' attribute

                # Extract all population data (there could be multiple columns for different years)
                population_cells = row.find_all('td', class_='rpop')

                # Create a dictionary to store population data for each year
                population_data = {}
                for i, cell in enumerate(population_cells):
                    population = cell.text.strip()
                    population_data[f'Population {years[i]}'] = convert_to_flt(population)

                # Append the extracted data into the list
                row_data = [name, status, area] + list(population_data.values())
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame
          

        elif section_id == "largecities":
            # import pdb; pdb.set_trace()
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', {'class': None}).text.strip()
                # Extract all population data (there could be multiple columns for different years)
                population_tag = row.find('td', {'class': 'rpop'}).text.strip()

                # Append the extracted data into the list
                row_data = [name, population_tag]
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame

        elif section_id == "citysection":
            # import pdb; pdb.set_trace()
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', class_='rname').text.strip()
                # status = row.find('td', class_='rstatus').text.strip()

                # Extract area and population data for different census years
                # import pdb; pdb.set_trace()
                area = row.find('td', class_='noviz').text.strip()  # Get the 'data-area' attribute

                # Extract all population data (there could be multiple columns for different years)
                population_cells = row.find_all('td', class_='rpop')

                # Create a dictionary to store population data for each year
                population_data = {}
                for i, cell in enumerate(population_cells):
                    population = cell.text.strip()
                    population_data[f'Population {years[i]}'] = convert_to_flt(population)

                # Append the extracted data into the list
                row_data = [name, area] + list(population_data.values())
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame

        
    
    if section_id == 'adminareas':
        columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    elif section_id == 'largecities':
        columns = ['Name'] + [year for year in years] 
    elif section_id == 'citysection':
        columns = ['Name', 'Area'] + [year for year in years] 

    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column



In [5]:
# test, year = make_df_city(page, 'citysection')
# test

In [11]:
page = requests.get('https://www.citypopulation.de/en/bangledesh/admin')
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE HTML>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   City Population: Error 404
  </title>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <script src="/js/cp_data_m.js">
  </script>
  <script src="/js/cp_menu.js">
  </script>
  <script>
   var pagename = 'error404'
  </script>
  <script src="/js/cp_cityindex_m.js">
  </script>
 </head>
 <body onload="start('en',false,false,false)">
  <script>
   writeMenu('en')
  </script>
  <div class="mobiadv">
   <script type="text/javascript">
    show_mobiadv(false)
   </script>
  </div>
  <div id="headline">
   <div id="orient">
    <a href="/">
     Home
    </a>
   </div>
  </div>
  <div class="cindex">
   <header>
    <img alt="Icon" class="headicon" src="/images/icons/settings.svg"/>
    <h1>
     Error 404
    </h1>
    <p>
     Sorry, the requested page does not exist.
    </p>
   </header>
   <h2>
    Important Links
   </h2>
   <div class="mcolboxes veryslimboxes">
    <ul>
     <li class="middle s2">
      <a href="

In [ ]:
page = requests.get('https://www.citypopulation.de/en/brazil/cities/')
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())
# df, latest_population_column = make_df_admin(page)


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="Brazil: States and Major Cities with population statistics, maps, charts, weather and web information." name="description"/>
  <title>
   Brazil: States and Major Cities - Population Statistics, Maps, Charts, Weather and Web Information
  </title>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <script src="/js/cp_data_m.js">
  </script>
  <script src="/js/cp_citypage_m.js">
  </script>
  <script src="/js/cp_menu.js">
  </script>
  <style>
   article.cpage { top: 469px }
    div#map { height: 369px }
    @media screen and (min-width: 1395px) {
      article.cpage { top: 99px; left: 720px }
      div#map { left: 160px; top: -1px; height: auto; bottom: -1px; width: 540px }
      header.cpage { left: 720px }
      div#headline { left: 702px }
    }
  </style>
 </head>
 <body itemscope="" itemtype="http://schema.org/Country" onload="start()">
  <script>
   writeMenu('en','htmlpage','brazil-cities')
  </

In [ ]:
tables = ['adminareas', 'largecities', "citysection"]

In [ ]:
for country in countries:
    print(country)
    if country not in city_country:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/admin/")
        df, year = make_df_admin(page)

    
    else:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/cities/")
        admin_df, year = make_df_city(page, 'adminareas')
        city_df, year = make_df_city(page, 'largecities')
        city_section_df, year = make_df_city(page, 'citysection')

        df = pd.concat([admin_df, city_df, city_section_df], axis=0)

        # country_admin_df_name = f"{country}_admin_df"
        # country_city_df_name = f"{country}_city_df"
        # country_city_section_df_name = f"{country}_city_section_df"
        # globals()[country_admin_df_name] = admin_df
        # globals()[country_city_df_name] = city_df
        # globals()[country_city_section_df_name] = city_section_df

# import pdb; pdb.set_trace()
        country_df_name = f"{country}_df"  
        country_year_name = f"{country}_year"
        globals()[country_df_name] = df
        globals()[country_year_name] = year
    
    


bangladesh
brazil
cambodia
china
indonesia
mexico
rwanda


C:\Users\Ethan.loh\AppData\Local\Temp\ipykernel_13740\4226508739.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([admin_df, city_df, city_section_df], axis=0)


kenya


In [13]:
page = requests.get(f"https://www.citypopulation.de/en/kenya/sub/admin/")
kenya_df, year = make_df_admin(page)
kenya_df

,Name,Status,Area,2009,2019
0,Baringo,County,10976.4,555561.0,666763.0
1,Bomet,County,2530.9,730129.0,875689.0
2,Bungoma,County,3023.9,1375063.0,1670570.0
3,Busia,County,1696.3,743946.0,893681.0
4,Elgeyo-Marakwet,County,3032.0,369998.0,454480.0
5,Embu,County,2820.7,516212.0,608599.0
6,Garissa,County,44736.0,623060.0,841353.0
7,Homa Bay,County,3152.5,963794.0,1131950.0
8,Isiolo,County,25350.6,143294.0,268002.0
9,Kajiado,County,21871.1,687312.0,1117840.0


In [14]:
data = {
    "Name": [
        "Bomi", "Bong", "Gbarpolu", "Grand Bassa", "Grand Cape Mount",
        "Grand Gedeh", "Grand Kru (Kru Coast & Sasstown)", "Lofa", "Margibi (Gibi & Marshall)",
        "Maryland", "Montserrado", "Nimba", "Rivercess", "River Gee", "Sinoe"
    ],
    "Abbr.": [
        "BM", "BG", "GP", "GB", "CM", "GG", "GK", "LO", "MG", "MY", "MO", "NI", "RI", "RG", "SI"
    ],
    "Capital": [
        "Tubmanburg", "Gbarnga", "Bopolu", "Buchanan", "Robertsport", "Zwedru", "Barclayville",
        "Voinjama", "Kakata", "Harper", "Bensonville", "Sanniquellie", "Cestos City", "Fish Town", "Greenville"
    ],
    "Area (km²)": [
        1942, 8769, 9685, 7192, 5160, 10480, 3894, 9909, 2615, 2296, 1998, 11546, 5113, 5110, 10133
    ],
    "Population (1984-02-01)": [
        66420, 187689, 48839, 159648, 79232, 103192, 62971, 199689, 151792, 62619, 491814, 313129, 47542, 39782, 64147
    ],
    "Population (2008-03-21)": [
        84119, 328919, 83288, 221693, 127076, 126146, 57931, 276863, 199689, 136404, 1182418, 462026, 71141, 66679, 102391
    ],
    "Population (2022-11-11)": [
        133705, 357859, 456761, 255803, 178060, 173903, 106342, 348998, 304946, 158817, 1925905, 621372, 101021, 124653, 151149
    ]
}

liberia_df = pd.DataFrame(data)
liberia_df

,Name,Abbr.,Capital,Area (km²),Population (1984-02-01),Population (2008-03-21),Population (2022-11-11)
0,Bomi,BM,Tubmanburg,1942,66420,84119,133705
1,Bong,BG,Gbarnga,8769,187689,328919,357859
2,Gbarpolu,GP,Bopolu,9685,48839,83288,456761
3,Grand Bassa,GB,Buchanan,7192,159648,221693,255803
4,Grand Cape Mount,CM,Robertsport,5160,79232,127076,178060
5,Grand Gedeh,GG,Zwedru,10480,103192,126146,173903
6,Grand Kru (Kru Coast & Sasstown),GK,Barclayville,3894,62971,57931,106342
7,Lofa,LO,Voinjama,9909,199689,276863,348998
8,Margibi (Gibi & Marshall),MG,Kakata,2615,151792,199689,304946
9,Maryland,MY,Harper,2296,62619,136404,158817


In [15]:
page = requests.get(f"https://www.citypopulation.de/en/rwanda/admin/")
rwanda_df, year = make_df_admin(page)
rwanda_df

,Name,Status,Area,2002,2012,2022
0,Est (Intara y'Iburasirazuba) [Eastern],Province,9458.0,1700137.0,2595703.0,3563145.0
1,Kigali,City,730.0,765325.0,1132686.0,1745555.0
2,Nord (Intara y'Amajyaruguru) [Northern],Province,3276.0,1560862.0,1726370.0,2038511.0
3,Ouest (Intara y'Iburengerazuba) [Western],Province,5882.0,2043555.0,2471239.0,2896484.0
4,Sud (Intara y'Amajyepfo) [Southern],Province,5963.0,2058674.0,2589975.0,3002699.0


In [16]:
page = requests.get(f"https://www.citypopulation.de/en/southafrica/admin/")
southafrica_df, year = make_df_admin(page)
southafrica_df

,Name,Status,Area,1996,2001,2011,2022
0,Eastern Cape,Province,168966.0,6147244.0,6278651.0,6562053.0,7230204.0
1,Free State (Oranje Free State),Province,129825.0,2633504.0,2706775.0,2745590.0,2964412.0
2,Gauteng,Province,18178.0,7834620.0,9390528.0,12272263.0,15099422.0
3,KwaZulu-Natal,Province,94361.0,8572302.0,9584129.0,10267300.0,12423907.0
4,Limpopo (Northern Transvaal),Province,125754.0,4576133.0,4995462.0,5404868.0,6572721.0
5,Mpumalanga (Eastern Transvaal),Province,76495.0,3124203.0,3365957.0,4039939.0,5143324.0
6,Northern Cape,Province,372889.0,1011864.0,991876.0,1145861.0,1355945.0
7,North West,Province,104882.0,2726828.0,2982064.0,3509953.0,3804548.0
8,Western Cape,Province,129462.0,3956875.0,4524335.0,5822734.0,7433020.0


In [17]:
page = requests.get(f"https://www.citypopulation.de/en/gambia/admin/")
gambia_df, year = make_df_admin(page)
gambia_df

,Name,Status,Area,1993,2003,2013
0,Banjul,Local Government Area,12.0,42326.0,35061.0,31054.0
1,Basse (Upper River),Local Government Area,2070.0,155059.0,182586.0,237220.0
2,Brikama (Western),Local Government Area,1764.0,234917.0,389594.0,688744.0
3,Janjanbureh,Local Government Area,1428.0,88247.0,107212.0,125204.0
4,Kanifeng,Local Government Area,76.0,228214.0,322735.0,377134.0
5,Kerewan (North Bank),Local Government Area,2255.0,156462.0,172835.0,220080.0
6,Kuntaur,Local Government Area,1467.0,67774.0,78491.0,96703.0
7,Mansakonko (Lower River),Local Government Area,1618.0,65146.0,72167.0,81042.0


In [18]:
page = requests.get(f"https://www.citypopulation.de/en/zambia/admin/")
zambia_df, year = make_df_admin(page)
zambia_df

,Name,Status,Area,2000,2010,2022
0,Central,Province,94394.0,1012257.0,1307111.0,2261336.0
1,Copperbelt,Province,31328.0,1581221.0,1972317.0,2768192.0
2,Eastern,Province,68948.8,1306173.0,1696555.0,2462682.0
3,Luapula,Province,50567.0,775353.0,991927.0,1519478.0
4,Lusaka,Province,21986.0,1391329.0,2191225.0,3093617.0
5,Muchinga,Province,70243.2,449296.0,607763.0,922213.0
6,Northern,Province,77650.0,809400.0,1105824.0,1623853.0
7,North-Western,Province,125826.0,583350.0,727044.0,1278357.0
8,Southern,Province,85283.0,1212124.0,1589926.0,2388091.0
9,Western,Province,126386.0,765088.0,902974.0,1375604.0


In [19]:
page = requests.get(f"https://www.citypopulation.de/en/ghana/admin/")
ghana_df, year = make_df_admin(page)
ghana_df

,Name,Status,Area,2010,2021
0,Ahafo (← Brong Ahafo),Region,5196.0,484210.0,564668.0
1,Ashanti,Region,24389.0,4780380.0,5440463.0
2,Bono (← Brong Ahafo),Region,11113.0,922617.0,1208649.0
3,Bono East (← Brong Ahafo),Region,23248.0,904156.0,1203400.0
4,Central,Region,9826.0,2201863.0,2859821.0
5,Eastern,Region,19323.0,2633154.0,2925653.0
6,Greater Accra,Region,3245.0,4010054.0,5455692.0
7,North East (← Northern),Region,9070.0,465005.0,658946.0
8,Northern,Region,26524.0,1544946.0,2310939.0
9,Oti (← Volta),Region,11066.0,636889.0,747248.0


In [20]:
data = {
    "Name": ["Hhohho", "Lubombo", "Manzini", "Shiselweni"],
    "Abbr.": ["HH", "LU", "MA", "SH"],
    "Capital": ["Mbabane", "Siteki", "Manzini", "Nhlangano"],
    "Area (km²)": [3569, 5947, 4068, 3779],
    "Population (1976-08-25)": [95759, 81800, 101277, 95735],
    "Population (1986-08-25)": [178936, 153958, 192596, 155569],
    "Population (1997-05-11)": [255455, 194323, 280972, 198978],
    "Population (2007-05-11)": [282734, 207731, 319530, 208454],
    "Population (2017-05-11)": [320651, 212531, 355945, 204111],
    "Population (2024-07-01 Projection)": [355700, 237500, 397000, 212200]
}

eswatini_df = pd.DataFrame(data)
eswatini_df

,Name,Abbr.,Capital,Area (km²),Population (1976-08-25),Population (1986-08-25),Population (1997-05-11),Population (2007-05-11),Population (2017-05-11),Population (2024-07-01 Projection)
0,Hhohho,HH,Mbabane,3569,95759,178936,255455,282734,320651,355700
1,Lubombo,LU,Siteki,5947,81800,153958,194323,207731,212531,237500
2,Manzini,MA,Manzini,4068,101277,192596,280972,319530,355945,397000
3,Shiselweni,SH,Nhlangano,3779,95735,155569,198978,208454,204111,212200


In [ ]:
with pd.ExcelWriter('data/africa_citypopulation_data.xlsx', engine='openpyxl') as writer:
    # bangladesh_df.to_excel(writer, sheet_name='Bangladesh', index=False)
    # brazil_df.to_excel(writer, sheet_name='Brazil', index=False)
    # cambodia_df.to_excel(writer, sheet_name='Cambodia', index=False)
    # china_df.to_excel(writer, sheet_name='China', index=False)
    # indonesia_df.to_excel(writer, sheet_name='Indonesia', index=False)
    # mexico_df.to_excel(writer, sheet_name='Mexico', index=False)
    kenya_df.to_excel(writer, sheet_name='Kenya', index=False)
    liberia_df.to_excel(writer, sheet_name='Liberia', index=False)
    rwanda_df.to_excel(writer, sheet_name='Rwanda', index=False)
    southafrica_df.to_excel(writer, sheet_name='South Africa', index=False)
    gambia_df.to_excel(writer, sheet_name='Gambia', index=False)
    zambia_df.to_excel(writer, sheet_name='Zambia', index=False)
    malawi_df.to_excel(writer, sheet_name='Malawi', index=False)
    ghana_df.to_excel(writer, sheet_name='Ghana', index=False)
    eswatini_df.to_excel(writer, sheet_name='Eswatini', index=False)
    workbook = writer.book

